In [18]:
# Imports

# pandas
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import math
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LogisticRegression
sns.set_style('whitegrid')
%matplotlib inline


In [3]:
# get titanic & test csv files as a DataFrame
titanic_df = pd.read_csv("train.csv", dtype={"Age": np.float64}, )
test_df    = pd.read_csv("test.csv", dtype={"Age": np.float64}, )

# preview the data
titanic_df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,NaN,S


In [4]:
titanic_df.columns

Index([u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex', u'Age',
       u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')

In [5]:
male = titanic_df[titanic_df.Sex == 'male']
female = titanic_df[titanic_df.Sex == 'female']
print "Male Survival Percent: " + str(male.Survived.mean())
print "Female Survival Percent: " + str(female.Survived.mean())

Male Survival Percent: 0.188908145581
Female Survival Percent: 0.742038216561


Men were more likely to die. Shocking /s

In [6]:
def Cov(xs, ys, meanx=None, meany=None):
    xs = np.asarray(xs)
    ys = np.asarray(ys)
    if meanx is None:
        meanx = np.mean(xs)
    if meany is None:
        meany =  np.mean(ys)
   
    cov = np.dot(xs-meanx, ys-meany) / len(xs)
    return cov

def Corr(xs, ys):
    xs = np.asarray(xs)
    ys = np.asarray(ys)
    meanx = np.mean(xs)
    varx = np.var(xs)
    meany = np.mean(ys)
    vary = np.var(ys)
    corr = Cov(xs, ys, meanx, meany) / math.sqrt(varx * vary)
    return corr

def SpearmanCorr(xs, ys):
    xranks = pd.Series(xs).rank()
    yranks = pd.Series(ys).rank()
    return Corr(xranks, yranks)


Here I recode a few of the categories (Sex and Embarked) into Binary data. The Embarked column has been split into 3 binary columns ('EmbarkedS','EmbarkedC','EmbarkedQ')

In [16]:
columns = ['Pclass', 'SexNum', 'Age', 'SibSp', 'Parch', 'Fare', 'EmbarkedS','EmbarkedC','EmbarkedQ']
oldNewMapSex = {'male': 1, 'female': 0}
oldNewMapS = {'S': 1, 'C': 0, 'Q': 0}
oldNewMapC = {'S': 0, 'C': 1, 'Q': 0}
oldNewMapQ = {'S': 0, 'C': 0, 'Q': 1}
titanic_df['SexNum'] = titanic_df['Sex'].map(oldNewMapSex)
titanic_df['EmbarkedS'] = titanic_df['Embarked'].map(oldNewMapS)
titanic_df['EmbarkedC'] = titanic_df['Embarked'].map(oldNewMapC)
titanic_df['EmbarkedQ'] = titanic_df['Embarked'].map(oldNewMapQ)

Here I compute and display the Pearson and Spearman Correlations for a variety or variables and survived. 

In [15]:
for column in columns:
    titanic_dfNoNA = titanic_df.dropna(subset=['Survived', column])
    print column
    
    print "Pearson Correlations: " + str(Corr(titanic_dfNoNA[column],titanic_dfNoNA.Survived))
    print "Spearman Correlations: " + str(SpearmanCorr(titanic_dfNoNA[column],titanic_dfNoNA.Survived))

Pclass
Pearson Correlations: -0.338481035961
Spearman Correlations: -0.33966793665
SexNum
Pearson Correlations: -0.543351380658
Spearman Correlations: -0.543351380658
Age
Pearson Correlations: -0.0772210945722
Spearman Correlations: -0.0525653000447
SibSp
Pearson Correlations: -0.0353224988857
Spearman Correlations: 0.0888794846809
Parch
Pearson Correlations: 0.0816294070835
Spearman Correlations: 0.138265632865
Fare
Pearson Correlations: 0.257306522385
Spearman Correlations: 0.323736139445
EmbarkedS
Pearson Correlations: -0.151777048594
Spearman Correlations: -0.151777048594
EmbarkedC
Pearson Correlations: 0.169965966813
Spearman Correlations: 0.169965966813
EmbarkedQ
Pearson Correlations: 0.00453572872399
Spearman Correlations: 0.00453572872399


Prep data for logistic regression

In [52]:
titanic_dfReg = titanic_df.drop(['PassengerId','Name','Ticket','Embarked','Sex','Cabin'], axis=1)
titanic_dfReg = titanic_dfReg.dropna()
predData = titanic_dfReg.drop("Survived",axis=1)
survived = titanic_dfReg["Survived"]

Run Logistic Regression and compute the score on the initial data

In [53]:
logreg = LogisticRegression()
logreg.fit(predData, survived)
logreg.score(predData, survived)

0.7963483146067416

Display the coefficeints for each variable from the logistic regression

In [49]:
coeffs = logreg.coef_[0]
name = titanic_dfReg.columns[1:]
for i in range(len(coeffs)):
    print name[i] + ": " + str(coeffs[i])

Pclass: -0.917313200045
Age: -0.0326142424397
SibSp: -0.307305272134
Parch: -0.0512627096075
Fare: 0.00366830420125
SexNum: -2.38470231233
EmbarkedS: 1.05092552022
EmbarkedC: 1.41775488857
EmbarkedQ: 0.510784246869
